In [4]:
import torch
import torch.nn as nn
import torch.nn.init as init
import random
import copy
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
import DeepTreeEnsemble
import torch.nn.functional as F
from importlib import reload
import pandas as pd
reload(DeepTreeEnsemble)
from DeepTreeEnsemble import DeepTreeEnsemble, create_dataloaders_for_dataset 
# if this ^ doesn't work, try 'pip install -e .' in the root directory of it. Make sure you are in 
# the desired environment. 

In [5]:
train_dataloader, test_dataloader, input_size, output_size = create_dataloaders_for_dataset(dataset_name='breast_cancer', task='classification', batch_size=32)

xtrain size 455
xtest size 114
y_train size 455
y_test size 114


In [6]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_layers, output_size, dropout=.5):
        super(MLP, self).__init__()
        self.layers = nn.ModuleList()
        self.batch_norms = nn.ModuleList()

        # Create the first hidden layer
        self.layers.append(nn.Linear(input_size, hidden_layers[0]))
        self.batch_norms.append(nn.BatchNorm1d(hidden_layers[0]))

        # Create additional hidden layers
        for i in range(1, len(hidden_layers)):
            self.layers.append(nn.Linear(hidden_layers[i-1], hidden_layers[i]))
            self.batch_norms.append(nn.BatchNorm1d(hidden_layers[i]))

        # Output layer
        self.output = nn.Linear(hidden_layers[-1], output_size)

        # Dropout layer
        self.dropout = nn.Dropout(dropout)


    def forward(self, x):
        i = 0
        for layer, batch_norm in zip(self.layers, self.batch_norms):
            # x = F.relu(batch_norm(layer(x)))
            i=i+1
            x = self.dropout(x)
            # print('Layer dtype: ', layer.weight.dtype)
            x = x.to(torch.float32) # <-- area of concern
            # print('tensor dytpe: ', x.dtype)
            # layer.weight = layer.weight.to(torch.float32)
            x = F.relu(batch_norm(layer(x)))
            
            # print(f'dtype is {x.dtype} after layer {i}')

        x = self.output(x)
        return x

In [7]:
hidden_layers = [25, 25]
multiLayerPerceptron = MLP(input_size=input_size, hidden_layers=hidden_layers, output_size=output_size+1, dropout=.5) #hidden size default at 100

# Fill DTE params
modelSaveDirectory = './saved_models/'
DTEmlp4b = DeepTreeEnsemble(task_name='classification',
                                model_arch=multiLayerPerceptron,
                                model_dir=modelSaveDirectory, 
                                base_number=4,
                                epochs=10,
                                train_dataloader=train_dataloader, 
                                test_dataloader=test_dataloader,
                                learning_rate=0.0001)

In [16]:
def run_experiment(DTE, trails, output_directory):
    results = pd.DataFrame()

    for i in range(trails):
        print(f'Trail {i}: \n')
        df1 = DTE.train_DTE()
        df2 = DTE.single_model()
        combined_df = pd.concat([df1, df2], axis=1)
        results = pd.concat([results, combined_df], axis=0)

    results.to_csv(output_directory, index=False)

In [13]:
output_directory = './output.csv'
run_experiment(DTEmlp4b, trails=5, output_directory=output_directory)

Randomly initializing 4 models
Training Model with id 1
in multi


Training Process: 100%|██████████| 10/10 [00:00<00:00, 11.96it/s]


Total Epochs: 10
Train loss start: 1.2131139039993286, Train loss end: 1.2066375017166138
Highest Mean Train Accuracy (over epoch): 0.26654761904761903
Test loss start: 1.12948739528656, Test loss end: 1.0565742254257202
Highest mean Test Accuracy (over epoch): 0.4325520833333334
 Test F1 score at highest accuracy: 0.3516226254041805
./saved_models/model_1.pth 

Training Model with id 2
in multi


Training Process: 100%|██████████| 10/10 [00:00<00:00, 13.25it/s]


Total Epochs: 10
Train loss start: 1.1202521324157715, Train loss end: 0.9130666851997375
Highest Mean Train Accuracy (over epoch): 0.4311309523809524
Test loss start: 1.1419830322265625, Test loss end: 1.1100441217422485
Highest mean Test Accuracy (over epoch): 0.18619791666666666
 Test F1 score at highest accuracy: 0.4421704066533522
./saved_models/model_2.pth 

Training Model with id 3
in multi


...

Total Epochs: 10
Train loss start: 0.9425079822540283, Train loss end: 0.912303626537323
Highest Mean Train Accuracy (over epoch): 0.719672619047619
Test loss start: 1.020691990852356, Test loss end: 0.9935900568962097
Highest mean Test Accuracy (over epoch): 0.6331597222222223
 Test F1 score at highest accuracy: 0.7353824875756874
./saved_models/model_8.pth 

Training Model with id 6
in multi


Training Process: 100%|██████████| 10/10 [00:00<00:00, 12.73it/s]


Total Epochs: 10
Train loss start: 0.8405287861824036, Train loss end: 0.9956708550453186
Highest Mean Train Accuracy (over epoch): 0.7883035714285714
Test loss start: 0.9595686197280884, Test loss end: 0.9508195519447327
Highest mean Test Accuracy (over epoch): 0.7885416666666666
 Test F1 score at highest accuracy: 0.7975634863438555
./saved_models/model_6.pth 

 Maximum accuracy: 0.7885416666666666, F1: 0.7975634863438555, Test: 0.9532636925578117, Train: 0.8564161062240601
in multi


Training Process: 100%|██████████| 70/70 [00:07<00:00,  9.29it/s]

Total Epochs: 70
Train loss start: 1.0115265846252441, Train loss end: 0.9010607004165649
Highest Mean Train Accuracy (over epoch): 0.7873511904761905
Test loss start: 1.0668747425079346, Test loss end: 0.8029728531837463
Highest mean Test Accuracy (over epoch): 0.8097346230158731
 Test F1 score at highest accuracy: 0.7793360362878065


In [14]:
hidden_layers = [25, 25]
multiLayerPerceptron = MLP(input_size=input_size, hidden_layers=hidden_layers, output_size=output_size+1, dropout=.5) #hidden size default at 100

# Fill DTE params
modelSaveDirectory = './saved_models/'
DTEmlp8b = DeepTreeEnsemble(task_name='classification',
                                model_arch=multiLayerPerceptron,
                                model_dir=modelSaveDirectory, 
                                base_number=4,
                                epochs=10,
                                train_dataloader=train_dataloader, 
                                test_dataloader=test_dataloader,
                                learning_rate=0.0001)

In [17]:
output_directory = './outputBase8.csv'
run_experiment(DTEmlp8b, trails=5, output_directory=output_directory)

Randomly initializing 4 models
Training Model with id 1
in multi


Training Process: 100%|██████████| 10/10 [00:00<00:00, 14.67it/s]


Total Epochs: 10
Train loss start: 1.0107442140579224, Train loss end: 0.7855032682418823
Highest Mean Train Accuracy (over epoch): 0.5669642857142857
Test loss start: 1.0562001466751099, Test loss end: 1.0480875968933105
Highest mean Test Accuracy (over epoch): 0.5153645833333333
 Test F1 score at highest accuracy: 0.579538665320994
./saved_models/model_1.pth 

Training Model with id 2
in multi


Training Process: 100%|██████████| 10/10 [00:00<00:00, 14.51it/s]


Total Epochs: 10
Train loss start: 1.0421847105026245, Train loss end: 0.8273233771324158
Highest Mean Train Accuracy (over epoch): 0.5752678571428571
Test loss start: 1.0396450757980347, Test loss end: 0.9987768530845642
Highest mean Test Accuracy (over epoch): 0.588107638888889
 Test F1 score at highest accuracy: 0.582947050487938
./saved_models/model_2.pth 

Training Model with id 3
in multi


...

Total Epochs: 10
Train loss start: 0.8769213557243347, Train loss end: 0.8088529706001282
Highest Mean Train Accuracy (over epoch): 0.7629761904761905
Test loss start: 0.9882746934890747, Test loss end: 0.9568731784820557
Highest mean Test Accuracy (over epoch): 0.7793692129629631
 Test F1 score at highest accuracy: 0.7577742650657835
./saved_models/model_8.pth 

Training Model with id 6
in multi


Training Process: 100%|██████████| 10/10 [00:00<00:00, 12.64it/s]


Total Epochs: 10
Train loss start: 0.8592209219932556, Train loss end: 0.7261942028999329
Highest Mean Train Accuracy (over epoch): 0.8125
Test loss start: 0.9437457919120789, Test loss end: 0.9286413192749023
Highest mean Test Accuracy (over epoch): 0.7870783730158731
 Test F1 score at highest accuracy: 0.799618728848535
./saved_models/model_6.pth 

 Maximum accuracy: 0.7870783730158731, F1: 0.799618728848535, Test: 0.941957950592041, Train: 0.8497970906893412
in multi


Training Process: 100%|██████████| 70/70 [00:07<00:00,  9.33it/s]

Total Epochs: 70
Train loss start: 1.232723355293274, Train loss end: 0.9599242210388184
Highest Mean Train Accuracy (over epoch): 0.5646683673469388
Test loss start: 1.1433565616607666, Test loss end: 0.8128991723060608
Highest mean Test Accuracy (over epoch): 0.6940104166666667
 Test F1 score at highest accuracy: 0.6346859292015002
